Projet 2: Clément et Paolo



Chargé la base des donées 

In [4]:
import pandas as pd
import re
path = "https://raw.githubusercontent.com/remijul/dataset/master/SMSSpamCollection"
data = pd.read_csv(path, header=None, sep='\t')
data.rename(columns={0:'flag', 1:'text'}, inplace=True)
data


,flag,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


On crée le BOG (bag of things) que tiendra toutes le differents mots

In [5]:
bog = data['flag'].str.cat(sep = " ") #la combinaison de toutes les strings des mails
bog = list(set(re.split(r"\W+",bog)))

Selection de target

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTENC

# preprocessing
path = "https://raw.githubusercontent.com/remijul/dataset/master/SMSSpamCollection"
data = pd.read_csv(path, names = ["flag", "text"], sep='\t')
data["len"] = data["text"].str.split(r"(?!^)").str.len()
data["nb"] = data["text"].str.split().str.len()
data.dropna(inplace= True)
data.drop_duplicates(inplace = True)
sm = SMOTENC(categorical_features = ["text"], random_state= 3)

# agroupement de encodage et modelisation

preparation = ColumnTransformer(
    transformers=[
        ('data_cat', CountVectorizer() , "text"),
        ('data_num', RobustScaler(), ["len","nb"])
    ])

model = Pipeline(steps=[
                       ('preparation', preparation),
                       ('modelisation', LogisticRegression())
                       ])


params = {
        'modelisation': [LogisticRegression(), KNeighborsClassifier(), DecisionTreeClassifier()]
}

X = data.drop(["flag"], axis = 1)
y = data["flag"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 3, stratify=y)

X_smot, y_smot = sm.fit_resample(X_train, y_train)

scorers = {"accuracy": "accuracy", "recall": make_scorer(recall_score, pos_label = "spam")}

grid = GridSearchCV(model, params, scoring=scorers, refit = "recall")

# diviser train et test, appel du pipeline et regarder la performance

grid.fit(X_smot,y_smot)
y_pred = grid.predict(X_test)

print("Score", grid.best_score_)
print(grid.best_params_)



Score 0.9981012658227849
{'modelisation': LogisticRegression()}
